In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="png", remove_last=False)
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,Architecture
15,model_4_9_8,0.998668,0.822326,0.981718,0.996370,0.996091,0.005591,1.188109,0.016738,0.013399,0.015069,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
17,model_4_9_7,0.998630,0.822591,0.982056,0.996319,0.996107,0.005751,1.186335,0.016429,0.013585,0.015007,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
18,model_4_9_6,0.998577,0.822877,0.982312,0.996257,0.996108,0.005971,1.184420,0.016195,0.013815,0.015005,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
19,model_4_9_5,0.998506,0.823185,0.982452,0.996180,0.996087,0.006269,1.182359,0.016067,0.014100,0.015083,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
20,model_4_9_4,0.998412,0.823516,0.982438,0.996084,0.996040,0.006664,1.180150,0.016079,0.014453,0.015266,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
21,model_4_9_3,0.998289,0.823869,0.982222,0.995965,0.995958,0.007180,1.177789,0.016277,0.014892,0.015584,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
22,model_4_9_2,0.998130,0.824245,0.981745,0.995818,0.995830,0.007848,1.175277,0.016714,0.015437,0.016076,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
23,model_4_9_1,0.997926,0.824643,0.980935,0.995634,0.995646,0.008704,1.172614,0.017456,0.016115,0.016786,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
25,model_4_8_9,0.997612,0.824652,0.997266,0.983891,0.994025,0.010020,1.172554,0.018448,0.034910,0.026679,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
26,model_4_8_8,0.997612,0.824520,0.997441,0.984336,0.994265,0.010024,1.173438,0.017268,0.033945,0.025607,"Hidden Size=[30, 21], regularizer=0.2, learnin..."
